In [177]:
from openai import OpenAI
import json
import os

GPT_MODEL_3_5 = "gpt-3.5-turbo-1106"
GPT_MODEL_4 = "gpt-4-0125-preview"

OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return (response.choices[0].message.content)

In [178]:
usecase_flow_conditions="""
Main flow
Step 1 Admin presses the tab "Lesson Management" on the navbar.
Step 2 The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3 Admin presses one Lesson on the Lesson list.
Step 4 The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5 Admin presses one Topic on the Topic list of selected Lesson.
Step 6 The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7 Admin presses one word on vocabulary word list.
Step 8 The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9 Admin presses button "Add new question".
Step 10 The system pop up dialog to choose type of question Admin want to create.
Step 11 Admin presses option add "Create questions for the selection test".
Step 12 The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',  a valid sentence must be below 250 characters and not contains any specical character. one textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13 Admin fills the field "sentence" with a complete sentense has a symbol '...', 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 14 Admin presses button 'Save' to save the question information.
Step 15 System save the question and redirect Admin to the "Word Information" page of selected word.
Alternative flow 2
At step 13 of the basic flow Admin field all the complusary field except field "sentence".
Step 14 Admin presses button 'Save'.
Step 15 System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 16 Admin re-fill the missing field.
Step 17 Admin presses button 'Save'.
Alternative flow 3
At step 13 of the basic flow Admin field all the complusary field except field 'correct answer'.
Step 14 Admin presses button 'Save'.
Step 15 System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'correct answer' field.
Step 16 Admin re-fill the missing field.
Step 17 Admin presses button 'Save'.
Alternative flow 4
At step 13 of the basic flow Admin field all the complusary field except field 'other option 1'.
Step 14 Admin presses button 'Save'.
Step 15 System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 1' field.
Step 16 Admin re-fill the missing field.
Step 17 Admin presses button 'Save'.
Alternative flow 5
At step 13 of the basic flow Admin field all the complusary field except field 'other option 2'.
Step 14 Admin presses button 'Save'.
Step 15 System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 2' field.
Step 16 Admin re-fill the missing field.
Step 17 Admin presses button 'Save'.
Alternative flow 6
At step 13 of the basic flow Admin field all the complusary field except field 'other option 3'.
Step 14 Admin presses button 'Save'.
Step 15 System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 3'field.
Step 16 Admin re-fill the missing field.
Step 17 Admin presses button 'Save'.
Alternative flow 8
At step 13 of the basic flow Admin fills all the complusary field and Admin fills in the 'sentence' field by an invalid sentence.
Step 14 System show notification that 'Sentence is invalid', prompt Admin to re-enter the "sentence" field.
Step 16 Admin re-enter the missing field.
Step 17 Admin presses button 'Save'.

"""

In [179]:
extracted_elements = """Elements and their condition: 

{'Sentence field': {'valid': "Complete sentence with a symbol '...', below 250 characters and does not contain any special character", 'invalid': 'Not a complete sentence, more than 250 characters, contains special characters, or missing', 'type': 'text field'}, 'Correct answer field': {'valid': 'Filled with the correct word form of the blank in the sentence', 'invalid': 'Not filled', 'type': 'text field'}, 'Other option 1 field': {'valid': 'Filled with a different wrong answer word', 'invalid': 'Not filled', 'type': 'text field'}, 'Other option 2 field': {'valid': 'Filled with a different wrong answer word', 'invalid': 'Not filled', 'type': 'text field'}, 'Other option 3 field': {'valid': 'Filled with a different wrong answer word', 'invalid': 'Not filled', 'type': 'text field'}}

"""

In [180]:
SYSTEM_PROMPT_gen_test_scenarios ="""
Given the provided use case specification and a list of element and their extracted conditions.
your task is to generate a comprehensive set of test scenarios that test all the conditions of given elements.
The test scenarios generated must have clear conditions descritions and mention conditions is tested but do not need example data. 
The positive test scenarios generated must verify the combination of valid elements using a minimum number of test scenarios. 
The negative test scenarios generated must verify only one given invalid condition at a time. If there is no invalid condition, you don't need to write negative test scenario.
For cases where the invalid condition does not have a corresponding output description, predict the possible outcome to test.

Return the test scenarios generated in json format.
The JSON format should follow the following structure:
{
  "Positive Test Scenarios": [{ "positive test scenario description": {"conditions": conditions tested}}],
  "Negative Test Scenarios": [{ "negative test scenario description": {"conditions": conditions tested}}],
  }
An example of a test scenario: 
{
  "Positive Test Scenarios": [{ "Verify that users can successfully create a new account with valid username and valid passowrd.": {"conditions":  username valid and password valid}}],
  "Negative Test Scenarios": []
  }
"""


In [181]:
promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_gen_test_scenarios},
    { "role": "user", "content":usecase_flow_conditions +  extracted_elements }
  ]

gpt_response = ask(promptExtractCondtition, client, model)
print(gpt_response)

In [ ]:
test_scenario = json.loads(gpt_response)

In [ ]:
filter_testscenario = []

for key in test_scenario["Positive Test Scenarios"]:
    # print(" ", key)
    for k in key:
        print(k)
        filter_testscenario.append(k)


Verify that Admin can successfully create a new question for the selection test with all valid fields


In [ ]:

for key in test_scenario["Negative Test Scenarios"]:
    # print(" ", key)
    for k in key:
        print(k)
        filter_testscenario.append(k)


Verify that the system shows a notification when the 'Sentence' field is not filled
Verify that the system shows a notification when the 'Sentence' field contains more than 250 characters
Verify that the system shows a notification when the 'Sentence' field contains special characters
Verify that the system shows a notification when the 'Sentence' field is not a complete sentence
Verify that the system shows a notification when the 'Correct answer' field is not filled
Verify that the system shows a notification when the 'Other option 1' field is not filled
Verify that the system shows a notification when the 'Other option 2' field is not filled
Verify that the system shows a notification when the 'Other option 3' field is not filled


In [ ]:
for i in filter_testscenario:
    print(i)

Verify that Admin can successfully create a new question for the selection test with all valid fields
Verify that the system shows a notification when the 'Sentence' field is not filled
Verify that the system shows a notification when the 'Sentence' field contains more than 250 characters
Verify that the system shows a notification when the 'Sentence' field contains special characters
Verify that the system shows a notification when the 'Sentence' field is not a complete sentence
Verify that the system shows a notification when the 'Correct answer' field is not filled
Verify that the system shows a notification when the 'Other option 1' field is not filled
Verify that the system shows a notification when the 'Other option 2' field is not filled
Verify that the system shows a notification when the 'Other option 3' field is not filled
